In [1]:
%load_ext autoreload
%autoreload 2
import time
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

# load local model file and initialize model class
from model import ModelClass

model = ModelClass() # set a few number of periods.

par = model.par
sol = model.sol
sim = model.sim

model.solve()
model.simulate()

In [2]:
means_data = pd.read_csv("Data/mean_matrix.csv")

In [3]:
assets = np.array(means_data["formue_2018_Mean"])
savings = np.array(means_data["pension_2018_Mean"])
hours = np.array(means_data["yearly_hours_Mean"])/par.full_time_hours

In [4]:
mean = np.concatenate([assets, 
                       savings, 
                       hours])

In [5]:
covariance_matrix = pd.read_csv("Data/variance_matrix.csv")

numeric_cols = covariance_matrix.select_dtypes(include=[np.number]).columns[1:]

row_mask = covariance_matrix["_NAME_"].str.startswith("yearly_hours")
col_mask = [col for col in covariance_matrix.columns if col.startswith("yearly_hours")]

covariance_matrix.loc[row_mask, numeric_cols] = covariance_matrix.loc[row_mask, numeric_cols].astype(float)

covariance_matrix.loc[row_mask, numeric_cols] /= par.full_time_hours

covariance_matrix[col_mask] /= par.full_time_hours

In [6]:
weights = np.linalg.inv(np.nan_to_num(covariance_matrix[numeric_cols].to_numpy(), nan=0))

In [7]:
def scale_params(theta, bounds):
    """
    Scale theta to [0,1] given the bounds.
    """
    scaled = []
    for val, (low, high) in zip(theta, bounds):
        scaled_val = (val - low) / (high - low)
        scaled.append(scaled_val)
    return np.array(scaled)

def unscale_params(scaled_theta, bounds):
    """
    Convert scaled_theta in [0,1] back to original bounds.
    """
    unscaled = []
    for val, (low, high) in zip(scaled_theta, bounds):
        unscaled_val = low + val * (high - low)
        unscaled.append(unscaled_val)
    return np.array(unscaled)

In [8]:

def moment_func(sim_data):
    # Compute age-averaged moments
    avg_a_by_age = np.mean(sim_data.a, axis=0)  # Length 70
    avg_s_by_age = np.mean(sim_data.s, axis=0)  # Length 70
    avg_h_by_age = np.mean(sim_data.h, axis=0)  # Length 70

    # Concatenate and return
    return np.concatenate((avg_a_by_age, avg_s_by_age, avg_h_by_age))


def simulate_moments(theta, theta_names, model):
        
    # 1. Update model parameters
    for i, name in enumerate(theta_names):
        setattr(model.par, name, theta[i])
    
    # 2. Solve and simulate the model
    model.solve()
    model.simulate()
    
    # 3. Return the expanded vector of simulated moments
    return moment_func(model.sim)

def obj_func(scaled_theta, theta_names, mom_data, W, model, bounds, do_print=False):
    start_time = time.time()  # Start timing

    theta = unscale_params(scaled_theta, bounds)

    if do_print: 
        print_str = ''
        for i, name in enumerate(theta_names):
            print_str += f'{name}={theta[i]:2.3f} '
        print(print_str)
    
    # Compute simulated moments
    mom_sim = simulate_moments(theta, theta_names, model)

    # Sum of squared errors over all 175 elements
    obj = (mom_data - mom_sim).T @ W @ (mom_data - mom_sim)

    end_time = time.time()  # End timing
    elapsed_time = end_time - start_time
    
    if do_print: 
        print(f"Error = {obj:.6f}, Time = {elapsed_time:.4f} seconds")

    return obj



In [9]:
theta_names = ("beta", "sigma", "gamma", "mu", "r_s", "r_a")

theta_init = np.array([0.976, 1.026, 4.723, 7.887, 0.015, 0.009])

# Original bounds
orig_bounds = [(0.0, 1.0),   # beta
               (0.1, 6.0),   # sigma
               (0.1, 10.0),  # gamma
               (0.0, 10.0),  # mu
               (0.0, 0.02),  # r_s
               (0.0, 0.1)]   # r_a

theta_init_scaled = scale_params(theta_init, orig_bounds)

In [ ]:
objective = lambda theta: obj_func(theta, theta_names, mean, weights, model, orig_bounds, do_print=True)

res = minimize(
    objective, 
    theta_init_scaled,
    method='trust-constr',
    bounds=[(0,1)] * len(theta_init_scaled),
    tol=1e-6,
    options={"maxiter":10000}
)


beta=0.976 sigma=1.026 gamma=4.723 mu=7.887 r_s=0.015 r_a=0.009 
Error = 34.682835, Time = 3.2824 seconds
beta=0.976 sigma=1.026 gamma=4.723 mu=7.887 r_s=0.015 r_a=0.009 
Error = 34.682841, Time = 3.3814 seconds
beta=0.976 sigma=1.026 gamma=4.723 mu=7.887 r_s=0.015 r_a=0.009 
Error = 34.682794, Time = 4.3700 seconds
beta=0.976 sigma=1.026 gamma=4.723 mu=7.887 r_s=0.015 r_a=0.009 
Error = 34.682833, Time = 4.3998 seconds
beta=0.976 sigma=1.026 gamma=4.723 mu=7.887 r_s=0.015 r_a=0.009 
Error = 34.682836, Time = 4.4243 seconds
beta=0.976 sigma=1.026 gamma=4.723 mu=7.887 r_s=0.015 r_a=0.009 
Error = 34.682833, Time = 4.3260 seconds
beta=0.976 sigma=1.026 gamma=4.723 mu=7.887 r_s=0.015 r_a=0.009 
Error = 34.682839, Time = 4.3071 seconds
beta=0.794 sigma=3.683 gamma=4.923 mu=7.001 r_s=0.014 r_a=0.015 
Error = 103.378382, Time = 4.3168 seconds
beta=0.794 sigma=3.683 gamma=4.923 mu=7.001 r_s=0.014 r_a=0.015 
Error = 103.378382, Time = 4.2029 seconds
beta=0.794 sigma=3.683 gamma=4.923 mu=7.001 

In [ ]:
# theta_final = np.array([0.97422567, 1.14726037, 4.64679725, 7.93130135, 0.01460564,
#        0.01033787])

In [ ]:
# beta=0.941 sigma=1.097 gamma=4.714 mu=7.888 r_s=0.011 r_a=0.011 


theta_names = ("beta", "sigma", "gamma", "mu", "r_s", "r_a")
# theta_init = np.array(res.x)
theta_final = unscale_params(res.x, orig_bounds)

for i, name in enumerate(theta_names):
    setattr(model.par, name, theta_final[i])

model.solve()
model.simulate()

In [ ]:
a_dict = {
    'hours': [np.mean(model.sim.h, axis=0), hours],
    'illiquid': [np.mean(model.sim.s, axis=0), savings],
    'liquid': [np.mean(model.sim.a, axis=0), assets]
}

# Define colors
simulated_color = "navy"  # Dark blue
empirical_color = "darkred"  # Dark red
ci_color = "lightcoral"  # Light red for confidence bands

for key, (simulated, empirical) in a_dict.items():
    plt.figure(figsize=(10, 5))
    
    x_vals = np.arange(len(empirical)) + par.start_age
    
    plt.plot(x_vals, simulated, label=f"Simulated {key.capitalize()}", marker="o", color=simulated_color)
    plt.plot(x_vals, empirical, label=f"Empirical {key.capitalize()}", linestyle="--", marker="s", color=empirical_color)

    # # 99.9% confidence interval
    # ci = std_dev
    # plt.fill_between(x_vals, empirical - ci, empirical + ci, color=ci_color, alpha=0.4, label="Empirical Standard Deviation")

    plt.xlabel("Age")
    plt.ylabel(key.capitalize())
    plt.title(f"Comparison of Simulated and Empirical {key.capitalize()}")
    plt.legend()
    plt.grid(True)
    plt.show()
